In [21]:
class Tensor:
    def __init__(self,data):
        self.data = data

    def add(self,other):
        result =[]
        for i in range (len(self.data)):  # 행
            row = []
            for j in range(len(self.data[0])): #열
                row.append(self.data[i][j]+other.data[i][j])
            result.append(row)
        return Tensor(result)

    def sub(self,other):
        result = []
        for i in range (len(self.data)):
            row = []
            for j in range (len(self.data[0])):
                row.append(self.data[i][j]-other.data[i][j])
            result.append(row)
        return Tensor(result)

    def mul(self,other):
        result = []
        for i in range(len(self.data)):
            row = []
            for j in range(len(self.data[0])):
                    row.append(self.data[i][j]*other.data[i][j])
            result.append(row)
        return Tensor(result)
    
    def matmul(self,other): #행렬곱
        result = []
        # 3중 for문 필요!
        for i in range(len(self.data)):           # A의 행
            row = []
            for j in range(len(other.data[0])):   # B의 열
                sum_val = 0
                for k in range(len(self.data[0])): # A의 열 = B의 행
                    sum_val += self.data[i][k] * other.data[k][j]
                row.append(sum_val)
            result.append(row)
        return Tensor(result)

In [30]:
import random

class Linear:
    def __init__(self, in_features, out_features):
        '''
        # in_features 입력계수
        # out_features 출력계수
        '''
        self.W = []

        for i in range(in_features): #행갯수
            row = []
            for j in range(out_features): #갯수
                row.append(random.uniform(-0.1,0.1))
            self.W.append(row)


    # b 편향
        self.b = []
        for j in range (out_features):
            self.b.append(0.0)
    def forward(self, x):
        self.x = x
        # x 저장 (backward에서 사용하기 위해서ㅗ)
        # x: Tensor, shape [batch, in_features]
        # self.W: list, shape [in_features, out_features]
        # self.b: list, shape [out_features]
        # 
        # 출력: Tensor, shape [batch, out_features]
        W_tensor = Tensor(self.W)
        b_tensor = Tensor(self.b)

        y = x.matmul(W_tensor)
        # Y = w*x 

        #bias 더하기
        for i in range(len(y.data)):
            for j in range(len(y.data[0])):
                y.data[i][j] += self.b[j]

        return y
    
    def backward(self, grad_output):
        # 1. W.T 만들기
        W_T = []
        for j in range(len(self.W[0])):
            row = []
            for i in range(len(self.W)):
                row.append(self.W[i][j])
            W_T.append(row)
        W_T_tensor = Tensor(W_T)
        
        # 2. grad_x = grad_output @ W.T
        grad_x = grad_output.matmul(W_T_tensor)
        
        # 3. x.T 만들기
        x_T = []
        for j in range(len(self.x.data[0])):
            row = []
            for i in range(len(self.x.data)):
                row.append(self.x.data[i][j])
            x_T.append(row)
        x_T_tensor = Tensor(x_T)
        
        # 4. grad_W = x.T @ grad_output
        grad_W_tensor = x_T_tensor.matmul(grad_output)
        self.grad_W = grad_W_tensor.data
        
        # 5. grad_b = sum(grad_output)
        self.grad_b = [0.0] * len(self.b)
        for i in range(len(grad_output.data)):
            for j in range(len(grad_output.data[0])):
                self.grad_b[j] += grad_output.data[i][j]
        
        return grad_x

In [31]:
example=Linear(3,2)
print(example.W)
print(example.b)


[[-0.038994817865077774, 0.07489882934506784], [-0.016693077678339635, 0.04388086887526929], [-0.019602112677375924, -0.08937663471986437]]
[0.0, 0.0]


In [32]:
# Linear(3, 2) 생성
layer = Linear(3, 2)

# 입력: [1, 3]
x = Tensor([[1.0, 2.0, 3.0]])

# Forward
y = layer.forward(x)

print(y.data)  # [1, 2] 크기 출력

[[-0.12464420776125702, 0.07862535319693548]]


In [34]:
# Tensor 클래스 import (Day 1에서 만든 것)

# Linear 생성
layer = Linear(3, 2)

# Forward
x = Tensor([[1.0, 2.0, 3.0]])
y = layer.forward(x)
print("Forward output:", y.data)

# Backward
grad_output = Tensor([[1.0, 1.0]])
grad_x = layer.backward(grad_output)

print("\nBackward:")
print("grad_x:", grad_x.data)
print("grad_W:", layer.grad_W)
print("grad_b:", layer.grad_b)

Forward output: [[-0.3763499484922124, -0.0522521570900647]]

Backward:
grad_x: [[0.011984533873220543, -0.12174897689911444, -0.06569622855242292]]
grad_W: [[1.0, 1.0], [2.0, 2.0], [3.0, 3.0]]
grad_b: [1.0, 1.0]
